Install Dependencies

In [1]:

%pip install transformers datasets torch torchvision

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: transformers in c:\users\oum1hc\appdata\local\programs\python\python312\lib\site-packages (4.46.3)




[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Code ViT với Hugging Face

In [4]:
import torch
from transformers import ViTForImageClassification, ViTFeatureExtractor
from datasets import load_dataset
from torchvision import transforms
from torch.utils.data import DataLoader

# 1. Tải tập dữ liệu Fashion MNIST
# Chỉ tải 10% dữ liệu để giảm thời gian chạy
train_dataset = load_dataset("fashion_mnist", split="train[:1%]")
test_dataset = load_dataset("fashion_mnist", split="test[:1%]")

# 2. Tiền xử lý dữ liệu với ViT Feature Extractor
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
transform = transforms.Compose([
    transforms.Grayscale(3),  # Chuyển từ grayscale sang RGB
    transforms.Resize((224, 224)),  # Resize ảnh về 224x224
    transforms.ToTensor(),  # Chuyển ảnh thành tensor
    transforms.Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
])

def preprocess_data(example):
    example["pixel_values"] = transform(example["image"])
    return example

# Áp dụng tiền xử lý cho tập dữ liệu
train_dataset = train_dataset.map(preprocess_data)
test_dataset = test_dataset.map(preprocess_data)

# 3. Tạo DataLoader để huấn luyện và đánh giá
def collate_fn(batch):
    # Chuyển đổi danh sách ảnh thành tensor
    pixel_values = torch.stack([torch.tensor(x["pixel_values"]) for x in batch])
    # Chuyển đổi danh sách nhãn thành tensor
    labels = torch.tensor([x["label"] for x in batch], dtype=torch.long)
    return {"pixel_values": pixel_values, "labels": labels}

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, collate_fn=collate_fn)

# 4. Tải mô hình Vision Transformer
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=10  # Fashion MNIST có 10 lớp
)

# Chuyển model sang GPU nếu có
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 5. Khai báo optimizer và loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
criterion = torch.nn.CrossEntropyLoss()

# 6. Huấn luyện mô hình
epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        pixel_values = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)

        # Forward pass
        outputs = model(pixel_values=pixel_values, labels=labels)
        loss = outputs.loss

        # Backward pass và cập nhật trọng số
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}")

# 7. Đánh giá mô hình
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch in test_loader:
        pixel_values = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(pixel_values=pixel_values)
        predictions = torch.argmax(outputs.logits, dim=-1)
        correct += (predictions == labels).sum().item()
        total += labels.size(0)

accuracy = correct / total
print(f"Test Accuracy: {accuracy * 100:.2f}%")


c:\Users\oum1hc\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Map: 100%|██████████| 100/100 [00:00<00:00, 189.24 examples/s]
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3, Loss: 2.0310
Epoch 2/3, Loss: 1.4253
Epoch 3/3, Loss: 0.9975
Test Accuracy: 82.00%
